In [1]:
import numpy as np
from pydgrid import grid

In [19]:
Z_trafo_b = 20.0e3**2/25.0e6
V_hv = 220.0/np.sqrt(3)
S1,pf1,S2,pf2 = 15300.0,0.98,5280,0.95
S_12_cplx = S1*pf1 + 1j*np.sin(np.arccos(pf1))*S1 + S2*pf2 + 1j*np.sin(np.arccos(pf2))*S2
pf_12 = np.cos(np.angle(S_12_cplx))

S1,pf1,S2,pf2 = 215.0,0.97,390.0,0.85
S_14_cplx = S1*pf1 + 1j*np.sin(np.arccos(pf1))*S1 + S2*pf2 + 1j*np.sin(np.arccos(pf2))*S2
pf_14 = np.cos(np.angle(S_14_cplx))

data = {     
        "buses":[
                 {"bus": "HV", "pos_x":   0, "pos_y":  0, "units": "m", "U_kV":220.0},
                 {"bus": "12", "pos_x":   0, "pos_y":  -5, "units": "m", "U_kV":20.0},
                 {"bus": "13", "pos_x":   0, "pos_y":  -25, "units": "m", "U_kV":20.0},
                 {"bus": "14", "pos_x":   0, "pos_y":  -85, "units": "m", "U_kV":20.0},
                ],
        "grid_formers":[
                        {"bus": "HV",
                        "bus_nodes": [1, 2, 3], "deg": [0, -120, -240],
                        "kV": [V_hv,V_hv,V_hv]}
                       ],
        "transformers":[
                        {"bus_j": "HV",  "bus_k": "12",  "S_n_kVA": 25.0e3, "U_j_kV":220, "U_k_kV":20,
                         "R_cc_pu": 0.19/Z_trafo_b, "X_cc_pu":1.91/Z_trafo_b, "connection": "Dyn1",   "conductors_j": 3, "conductors_k": 4}
                       ],
        "lines":[
                {"bus_j": "12",  "bus_k": "13",  "code": "1", "m": 4.89e3},
                {"bus_j": "13",  "bus_k": "14",  "code": "1", "m": 2.99e3},
                ],
        "shunts":[
                 #{"bus": "12" , "R": 0.001, "X": 0.0, "bus_nodes": [4,0]},
                 {"bus": "14" , "R": 1.0e-10, "X": 0.0, "bus_nodes": [1,0]},
                 #{"bus": "14" , "R": 1.0e-10, "X": 0.0, "bus_nodes": [2,3]},
                 ],
        "loads":[
                 #{"bus": "12" , "kVA":S_12_cplx, "pf": pf_12,"type":"3P"},
                 #{"bus": "13" , "kVA":40.0, "pf": 0.85,"type":"3P"},
                 #{"bus": "14" , "kVA":S_14_cplx, "pf":pf_14,"type":"3P"},
                ],
        "line_codes":
            {
            "1":  {"R1":0.510,"X1":0.366,"B1_mu":3.172},
            "2":  {"R1":89.0e-3,"X1":3.5e-3},
            }
       } 

In [20]:
sys1 = grid()
sys1.read(data)  # Load data
sys1.pf()  # solve power flow
sys1.get_v()      # post process voltages
sys1.get_i()      # post process currents

from pydgrid.plot_bokeh import plot_results
fig = plot_results(sys1)

In [8]:
S_12_cplx

(20010+4693.341029123436j)

In [9]:
pf_12

0.9735784929115807